In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os 
import cv2
import numpy as np
import math
import csv
from google.colab.patches import cv2_imshow

In [209]:
def rect_detect(img, th1=30, th2=120):
    """
    根据图像中轮廓个数与不同阀值比较判断图像类型 - 表格图(t),无表格图(n),虚线表格图(d)
    :params img:
            th1: 阀值1 用于区分虚线图和无表格图
            th2: 阀值2 用于区分无表格图和表格图
    :return type: -t / -n / -d
    Modify:
        21.06.2020 
    """
    img_copy = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    binary = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C , cv2.THRESH_BINARY, 15, -2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)) #横线
    eroded = cv2.erode(binary, kernel, iterations = 1)                   
    dilatedcol = cv2.dilate(eroded, kernel, iterations = 1)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 70)) #竖线
    eroded = cv2.erode(binary, kernel, iterations = 1)
    dilatedrow = cv2.dilate(eroded, kernel, iterations = 1)
    merge = cv2.add(dilatedcol, dilatedrow)  
    # cv2_imshow(merge)
    contours, hierarchy = cv2.findContours(merge, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) < th1:
        return '-d'
    elif th1 <= len(contours) and len(contours) < th2:
        return '-n'
    else:
        return '-t'

In [211]:
file_dir = '/content/drive/My Drive/MSc Project/data/rotated_image'
file_names = []
for root, dirs, files in os.walk(file_dir):
    file_names = files

for i, file_name in enumerate(file_names):
    image = cv2.imread('/content/drive/My Drive/MSc Project/data/rotated_image/{}'.format(file_names[i]))
    typeImg = rect_detect(image)
    file_name = file_name.split('.')[0] + typeImg + '.' + file_name.split('.')[1]
  
    #保存图片
    folder = os.path.exists('/content/drive/My Drive/MSc Project/data/classification_image')
    if not folder:
        os.makedirs('/content/drive/My Drive/MSc Project/data/classification_image')
    cv2.imwrite('/content/drive/My Drive/MSc Project/data/classification_image/{}'.format(file_name), image)